# 📈 Прогноз продаж из Excel-файла

Этот ноутбук извлекает данные о продажах и строит прогноз с помощью модели Holt-Winters.

## 📂 Импорт библиотек

In [39]:
import os
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm  # Импорт tqdm для отображения прогресса

## 📊 Загрузка и первичная обработка данных

In [40]:
# Путь к папке с данными
file_path = "data.xlsx"

df = pd.read_excel(file_path)

In [41]:
# Создание временной метки
df["Дата"] = pd.to_datetime(df["Год"].astype(str) + "-" + df["Месяц"].astype(str) + "-01")

# Создание полного диапазона дат
full_date_range = pd.date_range(start=df["Дата"].min(), end=df["Дата"].max(), freq="MS")

In [42]:
# Упорядочиваем данные и заполняем пропущенные месяцы нулями
df_full = (
    df.set_index("Дата")
    .groupby(["Группа контрагента", "Продает на МП", "Код", "Артикул", "Наименование", "Группа", "Цена, руб.", "Вес, кг.", "Вид сырья"])
    .apply(lambda x: x.reindex(full_date_range, fill_value=0))
)

# Сбрасываем индекс
df_full = df_full.drop(["Группа контрагента", "Продает на МП", "Код", "Артикул", "Наименование", "Группа", "Цена, руб.", "Вес, кг.", "Вид сырья"],axis=1).reset_index()
df_full = df_full.rename(columns={"level_9":"Дата"})

In [44]:
df_full.columns

Index(['Группа контрагента', 'Продает на МП', 'Код', 'Артикул', 'Наименование',
       'Группа', 'Цена, руб.', 'Вес, кг.', 'Вид сырья', 'Дата', 'Год',
       'Квартал', 'Месяц', 'Отгружено, шт.', 'Оборот, руб.'],
      dtype='object')

In [45]:
# Извлечение года, квартала и месяца из даты
df_full["Год"] = df_full["Дата"].dt.year
df_full["Квартал"] = ((df_full["Дата"].dt.month - 1) // 3 + 1)
df_full["Месяц"] = df_full["Дата"].dt.month

# Переупорядочиваем столбцы для удобства
df_full = df_full[["Группа контрагента", "Продает на МП", "Группа", "Код", "Артикул", "Наименование", "Дата", "Год", "Квартал", "Месяц", "Отгружено, шт.", "Цена, руб.", "Вес, кг.", "Вид сырья"]]

,Группа контрагента,Продает на МП,Группа,Код,Артикул,Наименование,Дата,Год,Квартал,Месяц,"Отгружено, шт.","Цена, руб.","Вес, кг.",Вид сырья
0,Маркетплейсы,да,10 Сад/10 Компостеры,1010100101,АП 821,Компостер «Агроном Премиум» 800л,2020-01-01,2020,1,1,0,1943.4,7.17020,ПП вторичный
1,Маркетплейсы,да,10 Сад/10 Компостеры,1010100101,АП 821,Компостер «Агроном Премиум» 800л,2020-02-01,2020,1,2,0,1943.4,7.17020,ПП вторичный
2,Маркетплейсы,да,10 Сад/10 Компостеры,1010100101,АП 821,Компостер «Агроном Премиум» 800л,2020-03-01,2020,1,3,0,1943.4,7.17020,ПП вторичный
3,Маркетплейсы,да,10 Сад/10 Компостеры,1010100101,АП 821,Компостер «Агроном Премиум» 800л,2020-04-01,2020,2,4,0,1943.4,7.17020,ПП вторичный
4,Маркетплейсы,да,10 Сад/10 Компостеры,1010100101,АП 821,Компостер «Агроном Премиум» 800л,2020-05-01,2020,2,5,0,1943.4,7.17020,ПП вторичный
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170077,Светофоры,нет,12 Тара/01 Ящики складные,1112010110,АП 493,Ящик складной с перфорированными стенками и кн...,2025-02-01,2025,1,2,0,428.7,1.67215,ПП вторичный
170078,Светофоры,нет,12 Тара/01 Ящики складные,1112010110,АП 493,Ящик складной с перфорированными стенками и кн...,2025-03-01,2025,1,3,1512,428.7,1.67215,ПП вторичный
170079,Светофоры,нет,12 Тара/01 Ящики складные,1112010110,АП 493,Ящик складной с перфорированными стенками и кн...,2025-04-01,2025,2,4,0,428.7,1.67215,ПП вторичный
170080,Светофоры,нет,12 Тара/01 Ящики складные,1112010110,АП 493,Ящик складной с перфорированными стенками и кн...,2025-05-01,2025,2,5,0,428.7,1.67215,ПП вторичный


## 🔮 Holt-Winters прогнозирование

In [46]:
from sklearn.metrics import mean_absolute_error
import numpy as np
import itertools

# Параметры для оптимизации
alphas = betas = gammas = np.arange(0.20, 1, 0.10)
abg = list(itertools.product(alphas, betas, gammas))

# Функция для оптимизации параметров
def tes_optimizer(ts, abg, step=6):
    """
    Оптимизация параметров тройного экспоненциального сглаживания.
    
    ts: pd.Series - временной ряд
    abg: list - комбинации параметров
    step: int - количество шагов для прогноза
    """
    best_alpha, best_beta, best_gamma, best_mae = None, None, None, float("inf")
    train = ts[:-step]  # Обучающая выборка
    test = ts[-step:]   # Тестовая выборка

    for comb in abg:
        try:
            model = ExponentialSmoothing(train, trend="add", seasonal="add", seasonal_periods=12)
            fitted_model = model.fit(smoothing_level=comb[0], smoothing_slope=comb[1], smoothing_seasonal=comb[2])
            y_pred = fitted_model.forecast(step)
            mae = mean_absolute_error(test, y_pred)

            if mae < best_mae:
                best_alpha, best_beta, best_gamma, best_mae = comb[0], comb[1], comb[2], mae

        except Exception as e:
            # Пропускаем некорректные комбинации
            continue

    return best_alpha, best_beta, best_gamma

In [47]:
# Группировка данных и прогнозирование
forecast_list = []
groups = list(df_full.groupby(["Группа контрагента", "Продает на МП", "Артикул"]))
total_groups = len(groups)  # Общее количество групп для прогноза
number_of_periods = 6

# Прогресс бар с использованием tqdm
for (group, mp, article), group_data in tqdm(groups, total=total_groups, desc="Прогнозирование"):
    # Упорядочиваем по дате
    group_data = group_data.sort_values("Дата")
    
    # Создаем временной ряд
    ts = group_data.set_index("Дата")["Отгружено, шт."]
    
    # Дата начала прогноза

    # Получаем текущую дату
    current_date = pd.Timestamp.now()
    
    # Устанавливаем день равным 1 и создаем новый Timestamp
    first_day_of_current_month = current_date.replace(day=1)

    # Устанавливаем день равным 1 и добавляем 1 месяц
    first_day_of_next_month = (current_date.replace(day=1) + pd.offsets.MonthBegin(1))

    start_date = first_day_of_next_month  #first_day_of_current_month  # Первый день текущего или следующего месяца

    forecast_index_future = pd.date_range(start=start_date, periods=number_of_periods, freq="MS")
    forecast_index_past = pd.date_range(end=start_date - pd.DateOffset(months=1), periods=number_of_periods, freq="MS")
    
    try:
        if len(ts) >= 24+number_of_periods:  # Достаточно данных для полной модели
            
            # Итеративное прогнозирование (включает предыдущие прогнозы)
            forecast_values_future = []
            history = ts.copy()
            for _ in range(number_of_periods):
                model = ExponentialSmoothing(history, trend="add", seasonal="add", seasonal_periods=12)
                fitted_model = model.fit() 
                next_forecast = fitted_model.forecast(1)[0]
                forecast_values_future.append(next_forecast)
                
                # Обновляем историю с новым прогнозом
                new_index = history.index[-1] + pd.offsets.MonthBegin()
                history = pd.concat([history, pd.Series(next_forecast, index=[new_index])])

            # Прогноз назад с обрезанными последними number_of_periods месяцами
            truncated_history = ts.iloc[:-number_of_periods].copy()  # Обрезаем последние месяцы
            forecast_values_past = []
            for i in range(number_of_periods):
                model = ExponentialSmoothing(truncated_history, trend="add", seasonal="add", seasonal_periods=12)
                fitted_model = model.fit()
                prev_forecast = fitted_model.forecast(1)[0]
                forecast_values_past.append(prev_forecast)
                
                # Вместо предсказанного значения добавляем фактическое, которое убрали ранее
                actual_value = ts.iloc[-number_of_periods + i]  # Берем фактическое значение из исходного ряда
                new_index = ts.index[-number_of_periods + i]  # Его индекс
                # Добавляем фактическое значение в начало истории
                truncated_history = pd.concat([truncated_history, pd.Series(actual_value, index=[new_index])])

        elif len(ts) >= 18+number_of_periods:  # Промежуточный вариант, назад - меньше 24

            # Итеративное прогнозирование (включает предыдущие прогнозы)
            forecast_values_future = []
            history = ts.copy()
            for _ in range(number_of_periods):
                model = ExponentialSmoothing(history, trend="add", seasonal="add", seasonal_periods=12)
                fitted_model = model.fit() 
                next_forecast = fitted_model.forecast(1)[0]
                forecast_values_future.append(next_forecast)
                
                # Обновляем историю с новым прогнозом
                new_index = history.index[-1] + pd.offsets.MonthBegin()
                history = pd.concat([history, pd.Series(next_forecast, index=[new_index])])

            # Прогноз назад с обрезанными последними number_of_periods месяцами
            truncated_history = ts.iloc[:-number_of_periods].copy()
            forecast_values_past = []
            for i in range(number_of_periods):
                model = ExponentialSmoothing(truncated_history, trend="add", seasonal=None)
                fitted_model = model.fit()
                prev_forecast = fitted_model.forecast(1)[0]
                forecast_values_past.append(prev_forecast)
                
                # Вместо предсказанного значения добавляем фактическое, которое убрали ранее
                actual_value = ts.iloc[-number_of_periods + i]  # Берем фактическое значение из исходного ряда
                new_index = ts.index[-number_of_periods + i]  # Его индекс
                # Добавляем фактическое значение в начало истории
                truncated_history = pd.concat([truncated_history, pd.Series(actual_value, index=[new_index])])
        
        elif len(ts) >= 12+number_of_periods:  # Достаточно данных для трендовой модели

            # Итеративное прогнозирование (включает предыдущие прогнозы)
            forecast_values_future = []
            history = ts.copy()
            for _ in range(number_of_periods):
                model = ExponentialSmoothing(history, trend="add", seasonal=None)
                fitted_model = model.fit() 
                next_forecast = fitted_model.forecast(1)[0] 
                forecast_values_future.append(next_forecast)
                
                # Обновляем историю с новым прогнозом
                new_index = history.index[-1] + pd.offsets.MonthBegin()
                history = pd.concat([history, pd.Series(next_forecast, index=[new_index])])

            # Прогноз назад с обрезанными последними number_of_periods месяцами
            truncated_history = ts.iloc[:-number_of_periods].copy()
            forecast_values_past = []
            for i in range(number_of_periods):
                model = ExponentialSmoothing(truncated_history, trend="add", seasonal=None)
                fitted_model = model.fit()
                prev_forecast = fitted_model.forecast(1)[0]
                forecast_values_past.append(prev_forecast)
                
                # Вместо предсказанного значения добавляем фактическое, которое убрали ранее
                actual_value = ts.iloc[-number_of_periods + i]  # Берем фактическое значение из исходного ряда
                new_index = ts.index[-number_of_periods + i]  # Его индекс
                # Добавляем фактическое значение в начало истории
                truncated_history = pd.concat([truncated_history, pd.Series(actual_value, index=[new_index])])

        elif len(ts) >= 6+number_of_periods:  # Промежуточный вариант, назад - меньше 12

            # Итеративное прогнозирование (включает предыдущие прогнозы)
            forecast_values_future = []
            history = ts.copy()
            for _ in range(number_of_periods):
                model = ExponentialSmoothing(history, trend="add", seasonal=None)
                fitted_model = model.fit() 
                next_forecast = fitted_model.forecast(1)[0] 
                forecast_values_future.append(next_forecast)
                
                # Обновляем историю с новым прогнозом
                new_index = history.index[-1] + pd.offsets.MonthBegin()
                history = pd.concat([history, pd.Series(next_forecast, index=[new_index])])

            # Прогноз назад с обрезанными последними number_of_periods месяцами
            truncated_history = ts.iloc[:-number_of_periods].copy()
            forecast_values_past = []
            for i in range(number_of_periods):
                weighted_mean = (truncated_history * range(1, len(truncated_history) + 1)).sum() / sum(range(1, len(truncated_history) + 1))
                forecast_values_past.append(weighted_mean)
                
                # Вместо предсказанного значения добавляем фактическое, которое убрали ранее
                actual_value = ts.iloc[-number_of_periods + i]  # Берем фактическое значение из исходного ряда
                new_index = ts.index[-number_of_periods + i]  # Его индекс
                # Добавляем фактическое значение в начало истории
                truncated_history = pd.concat([truncated_history, pd.Series(actual_value, index=[new_index])])

        else:
            forecast_values_future = [].copy()
            history = ts.copy()
            for _ in range(number_of_periods):
                weights = range(1, len(history) + 1)
                weighted_mean = (history * weights).sum() / sum(weights)
                forecast_values_future.append(weighted_mean)
                history = history.append(pd.Series(weighted_mean, index=[history.index[-1] + pd.offsets.MonthBegin()]))
            
            truncated_history = ts.iloc[:-number_of_periods]
            forecast_values_past = []
            for i in range(number_of_periods):
                weighted_mean = (truncated_history * range(1, len(truncated_history) + 1)).sum() / sum(range(1, len(truncated_history) + 1))
                forecast_values_past.append(weighted_mean)
                
                # Вместо предсказанного значения добавляем фактическое, которое убрали ранее
                actual_value = ts.iloc[-number_of_periods + i]  # Берем фактическое значение из исходного ряда
                new_index = ts.index[-number_of_periods + i]  # Его индекс
                # Добавляем фактическое значение в начало истории
                truncated_history = pd.concat([truncated_history, pd.Series(actual_value, index=[new_index])])
        
    except Exception as e:
        print(f"Ошибка при обработке группы {group}, артикула {article}: {e}")
        # Заполняем нулями в случае ошибки
        forecast_values_future = [0] * number_of_periods
        forecast_values_past = [0] * number_of_periods
    
    # Создаем DataFrame для прогноза
    forecast_future_df = pd.DataFrame({
        "Дата": forecast_index_future,
        "Год": forecast_index_future.year,
        "Квартал": ((forecast_index_future.month - 1) // 3 + 1),
        "Месяц": forecast_index_future.month,
        "Группа контрагента": group,
        "Продает на МП": mp,
        "Код": group_data["Код"].iloc[0],
        "Артикул": article,
        "Наименование": group_data["Наименование"].iloc[0],
        "Группа": group_data["Группа"].iloc[0],
        "Отгружено, шт.": forecast_values_future,
        "Цена, руб.": group_data["Цена, руб."].iloc[0],
        "Вес, кг.": group_data["Вес, кг."].iloc[0],
        "Вид сырья": group_data["Вид сырья"].iloc[0],
        "История/Прогноз": "Прогноз вперед"
    })

    forecast_past_df = pd.DataFrame({
        "Дата": forecast_index_past,
        "Год": forecast_index_past.year,
        "Квартал": ((forecast_index_past.month - 1) // 3 + 1),
        "Месяц": forecast_index_past.month,
        "Группа контрагента": group,
        "Продает на МП": mp,
        "Код": group_data["Код"].iloc[0],
        "Артикул": article,
        "Наименование": group_data["Наименование"].iloc[0],
        "Группа": group_data["Группа"].iloc[0],
        "Отгружено, шт.": forecast_values_past,
        "Цена, руб.": group_data["Цена, руб."].iloc[0],
        "Вес, кг.": group_data["Вес, кг."].iloc[0],
        "Вид сырья": group_data["Вид сырья"].iloc[0],
        "История/Прогноз": "Прогноз назад"
        
    })
    
    # Добавляем прогноз в список
    forecast_list.append(forecast_future_df)
    forecast_list.append(forecast_past_df)

Прогнозирование: 100%|█████████████████████████████████████████████████████████████| 2577/2577 [14:55<00:00,  2.88it/s]


## 💾 Сохранение прогноза для дальнейшей визуализации

In [48]:
# Объединяем прогнозы
forecast_result = pd.concat(forecast_list, ignore_index=True)

# Добавляем к исходным данным
df["История/Прогноз"] = "История"
result_df = pd.concat([df, forecast_result], ignore_index=True)

# Сортируем по дате
result_df = result_df.sort_values(["Группа контрагента", "Продает на МП", "Артикул", "Дата"])

In [50]:
# Сохранение

# Сортируем по дате
result_df = result_df.sort_values(["Группа контрагента", "Продает на МП", "Артикул", "Дата"])

# Сохранение результата
output_path = "Прогноз_с_историей.xlsx"
result_df.to_excel(output_path, index=False)

print(f"Прогноз успешно добавлен. Данные сохранены в {output_path}")

Прогноз успешно добавлен. Данные сохранены в Прогноз_с_историей.xlsx
